In [24]:
import tensorflow as tf
tf.__version__


'2.13.0'

In [25]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [26]:
tf.keras.backend.clear_session()

In [27]:
def dfToInputRNN(df_input):
    reshaped_df_input = df_input.to_numpy()
    reshaped_df_input = reshaped_df_input.reshape(reshaped_df_input.shape[0], 1, reshaped_df_input.shape[1])
    return reshaped_df_input

In [28]:
subject = 'Santa Catarina - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [29]:
data = pd.read_csv('../../Inputs Atuais/2008_01_model_input_SC.csv')
data = data[[col for col in data.columns if col != subject] + [subject]]
data = data.drop('Santa Catarina - Produção de Cimento (t)', axis=1)
data

,Ano,Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado,Santa Catarina - Desemprego,Santa Catarina - IDH,Santa Catarina - IDH Longevidade,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Santa Catarina - IDH Educacao,Santa Catarina - IDH Renda,Santa Catarina - Consumo de Cimento (t)
0,2008,1.404443e+08,6.811481e+06,20.042997,1.249726e+08,8.020915,0.802185,0.868622,5.483335e+08,2.546644e+09,...,22.705450,1015.709740,1159.562545,20.995392,16.887106,78.404177,2.205136,0.764435,0.777491,2653.587000
1,2009,1.414019e+08,6.842789e+06,20.044049,1.250753e+08,8.049936,0.802866,0.870023,5.302764e+08,2.679945e+09,...,22.246273,1015.468602,1144.441684,20.980939,16.993726,78.934917,2.172407,0.766524,0.776296,2669.400000
2,2010,1.450816e+08,6.897548e+06,20.092849,1.259937e+08,8.013741,0.803485,0.870869,5.046425e+08,2.891930e+09,...,21.990797,1015.572649,1157.010200,20.848998,16.800270,78.648476,2.054779,0.768517,0.775508,2852.759000
3,2011,1.483956e+08,6.888268e+06,20.118967,1.266496e+08,8.000656,0.804044,0.871172,4.854622e+08,3.091988e+09,...,21.342050,1015.696428,1175.275698,20.915915,16.747918,78.133882,1.801706,0.770407,0.775118,3087.810000
4,2012,1.513674e+08,6.873501e+06,20.117079,1.271489e+08,4.760238,0.804247,0.870803,4.867661e+08,3.189506e+09,...,20.883838,1015.789692,1187.018145,21.084508,16.909840,78.138320,1.427605,0.771704,0.774917,3268.310000
5,2013,1.539868e+08,6.837970e+06,20.102531,1.275421e+08,5.187787,0.805268,0.870203,5.225848e+08,3.093872e+09,...,21.423516,1015.883075,1188.987224,21.299307,17.182298,78.463284,1.150329,0.774346,0.775826,3466.227000
6,2014,1.560820e+08,6.814799e+06,20.085277,1.277386e+08,5.511795,0.805624,0.868970,5.676413e+08,2.779340e+09,...,21.963483,1015.872526,1152.319790,21.397543,17.331617,78.730613,1.252238,0.775791,0.776479,3246.749286
7,2015,1.574146e+08,6.792567e+06,20.038056,1.276578e+08,5.792915,0.805509,0.867217,5.179392e+08,2.561731e+09,...,21.813817,1015.950998,1142.708354,21.308544,17.138396,78.237612,1.540937,0.776440,0.777102,3017.084229
8,2016,1.585019e+08,6.760156e+06,20.026681,1.277375e+08,5.798844,0.807604,0.875108,4.524144e+08,2.516859e+09,...,22.023495,1016.049222,1167.145418,21.317634,16.960969,77.364404,1.763965,0.782298,0.769848,2776.217486
9,2017,1.591962e+08,6.740058e+06,20.047554,1.279508e+08,5.526597,0.807023,0.872664,4.213689e+08,2.497710e+09,...,21.801864,1016.030235,1163.903842,21.473728,17.031652,76.967201,1.872343,0.782053,0.770470,2891.489000


In [30]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado,Santa Catarina - Desemprego,Santa Catarina - IDH,Santa Catarina - IDH Longevidade,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Santa Catarina - IDH Educacao,Santa Catarina - IDH Renda
0,1.404443e+08,6.811481e+06,20.042997,1.249726e+08,8.020915,0.802185,0.868622,5.483335e+08,2.546644e+09,1.874883e+10,...,11.760983,22.705450,1015.709740,1159.562545,20.995392,16.887106,78.404177,2.205136,0.764435,0.777491
1,1.414019e+08,6.842789e+06,20.044049,1.250753e+08,8.049936,0.802866,0.870023,5.302764e+08,2.679945e+09,2.055015e+10,...,9.472585,22.246273,1015.468602,1144.441684,20.980939,16.993726,78.934917,2.172407,0.766524,0.776296
2,1.450816e+08,6.897548e+06,20.092849,1.259937e+08,8.013741,0.803485,0.870869,5.046425e+08,2.891930e+09,2.342247e+10,...,9.332485,21.990797,1015.572649,1157.010200,20.848998,16.800270,78.648476,2.054779,0.768517,0.775508
3,1.483956e+08,6.888268e+06,20.118967,1.266496e+08,8.000656,0.804044,0.871172,4.854622e+08,3.091988e+09,2.604265e+10,...,10.985689,21.342050,1015.696428,1175.275698,20.915915,16.747918,78.133882,1.801706,0.770407,0.775118
4,1.513674e+08,6.873501e+06,20.117079,1.271489e+08,4.760238,0.804247,0.870803,4.867661e+08,3.189506e+09,2.708753e+10,...,8.159014,20.883838,1015.789692,1187.018145,21.084508,16.909840,78.138320,1.427605,0.771704,0.774917
5,1.539868e+08,6.837970e+06,20.102531,1.275421e+08,5.187787,0.805268,0.870203,5.225848e+08,3.093872e+09,2.523394e+10,...,7.913561,21.423516,1015.883075,1188.987224,21.299307,17.182298,78.463284,1.150329,0.774346,0.775826
6,1.560820e+08,6.814799e+06,20.085277,1.277386e+08,5.511795,0.805624,0.868970,5.676413e+08,2.779340e+09,1.954361e+10,...,10.350442,21.963483,1015.872526,1152.319790,21.397543,17.331617,78.730613,1.252238,0.775791,0.776479
7,1.574146e+08,6.792567e+06,20.038056,1.276578e+08,5.792915,0.805509,0.867217,5.179392e+08,2.561731e+09,1.327596e+10,...,12.454398,21.813817,1015.950998,1142.708354,21.308544,17.138396,78.237612,1.540937,0.776440,0.777102
8,1.585019e+08,6.760156e+06,20.026681,1.277375e+08,5.798844,0.807604,0.875108,4.524144e+08,2.516859e+09,1.011370e+10,...,13.111908,22.023495,1016.049222,1167.145418,21.317634,16.960969,77.364404,1.763965,0.782298,0.769848
9,1.591962e+08,6.740058e+06,20.047554,1.279508e+08,5.526597,0.807023,0.872664,4.213689e+08,2.497710e+09,8.456309e+09,...,9.479812,21.801864,1016.030235,1163.903842,21.473728,17.031652,76.967201,1.872343,0.782053,0.770470


In [31]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0     2669.400000
1     2852.759000
2     3087.810000
3     3268.310000
4     3466.227000
5     3246.749286
6     3017.084229
7     2776.217486
8     2891.489000
9     2976.485000
10    3258.027000
11    3537.032000
12    4089.333000
13    4255.808000
14            NaN
Name: Santa Catarina - Consumo de Cimento (t), dtype: float64

In [32]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado,Santa Catarina - Desemprego,Santa Catarina - IDH,Santa Catarina - IDH Longevidade,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Santa Catarina - IDH Educacao,Santa Catarina - IDH Renda
0,-1.788831,-0.053289,-0.765812,-1.946661,1.383658,-1.752749,-0.621103,1.204752,-0.990234,0.166091,...,0.984445,1.435761,-0.708446,-0.446366,-0.967915,-0.373775,0.579127,1.332149,-1.644452,1.173188
1,-1.646154,0.543626,-0.732922,-1.849054,1.406794,-1.314663,-0.021470,0.820657,-0.546688,0.428911,...,-0.051710,0.887657,-1.954083,-1.451705,-1.034435,0.196789,1.104304,1.233818,-1.312944,0.756047
2,-1.097922,1.587678,0.792736,-0.976209,1.377939,-0.916498,0.340864,0.275392,0.158672,0.847995,...,-0.115145,0.582703,-1.416611,-0.616063,-1.641666,-0.838463,0.820866,0.880425,-0.996740,0.480739
3,-0.604178,1.410733,1.609275,-0.352824,1.367508,-0.556764,0.470555,-0.132596,0.824346,1.230291,...,0.633403,-0.191684,-0.777213,0.598353,-1.333696,-1.118619,0.311667,0.120113,-0.696766,0.344683
4,-0.161412,1.129186,1.550236,0.121740,-1.215809,-0.426008,0.312334,-0.104860,1.148829,1.382743,...,-0.646476,-0.738636,-0.295440,1.379072,-0.557774,-0.252116,0.316058,-1.003809,-0.491049,0.274475
5,0.228858,0.451761,1.095422,0.495478,-0.874960,0.230528,0.055613,0.657046,0.830615,1.112297,...,-0.757614,-0.094441,0.186945,1.509990,0.430797,1.205911,0.637615,-1.836837,-0.071805,0.592030
6,0.541016,0.009964,0.555991,0.682235,-0.616655,0.459321,-0.472188,1.615453,-0.215960,0.282052,...,0.345772,0.550099,0.132453,-0.927914,0.882911,2.004971,0.902142,-1.530670,0.157572,0.819692
7,0.739557,-0.413904,-0.920302,0.605403,-0.392541,0.385546,-1.222320,0.558230,-0.940033,-0.632425,...,1.298413,0.371448,0.537815,-1.566949,0.473308,0.970972,0.414308,-0.663324,0.260600,1.037237
8,0.901552,-1.031862,-1.275923,0.681176,-0.387815,1.733415,2.155314,-0.835563,-1.089341,-1.093812,...,1.596124,0.621735,1.045202,0.057796,0.515145,0.021497,-0.449746,0.006727,1.190164,-1.494981
9,1.004993,-1.415041,-0.623358,0.883876,-0.604855,1.359405,1.108906,-1.495939,-1.153056,-1.335633,...,-0.048438,0.357181,0.947123,-0.157727,1.233540,0.399746,-0.842785,0.332330,1.151189,-1.277683


In [33]:
reshaped_train_input = dfToInputRNN(train_input)

In [34]:
validation_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index - 1:-1]
validation_input = dfToInputRNN(validation_input)
validation_input

array([[[ 0.85831341, -0.98039984, -0.61699777,  0.76318959,
         -0.69841501,  0.11397466, -1.1290481 , -1.0453709 ,
          1.21859064, -0.99976018,  0.79428371,  1.15575749,
         -0.52559858,  1.30260559, -0.55253046, -0.25130359,
          1.38787176,  1.47154451, -1.80649616,  1.09296291,
          0.94281138,  0.52836084,  0.61258103,  0.87656257,
          1.47801536,  1.71982086,  1.28221181,  1.39086472,
          1.58209844,  1.08872091,  0.55212962, -1.81398118,
          1.44113674,  1.11191094,  1.30470835,  0.86390516,
         -1.67365968, -1.70738939, -1.55813196, -0.48498301,
         -0.75714639,  0.40770893, -0.27394734,  2.02323219,
          1.2908951 ,  1.70465513,  1.81148084,  1.39968653,
         -1.36191442,  1.28024593,  1.11054795,  0.12662799,
          1.40414112,  1.40468999,  1.45643959,  1.58509336,
          1.32283157,  1.30222047,  1.55138602,  1.91801602,
         -0.77489784,  2.32098884, -1.1942541 , -1.17081042,
         -1.68003402,  1

In [35]:
# Alvo para treinamento
train_target = target_data.iloc[:-3]
train_target

0     2669.400000
1     2852.759000
2     3087.810000
3     3268.310000
4     3466.227000
5     3246.749286
6     3017.084229
7     2776.217486
8     2891.489000
9     2976.485000
10    3258.027000
11    3537.032000
Name: Santa Catarina - Consumo de Cimento (t), dtype: float64

In [36]:
# Alvo para validação do treinamento
validation_target = target_data.iloc[split_index - 1:-3]
validation_target

11    3537.032
Name: Santa Catarina - Consumo de Cimento (t), dtype: float64

In [37]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[split_index:]
test_input

,Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado,Santa Catarina - Desemprego,Santa Catarina - IDH,Santa Catarina - IDH Longevidade,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Santa Catarina - IDH Educacao,Santa Catarina - IDH Renda
12,0.569016,-0.799091,-1.589659,0.368609,-0.769661,-1.10113,-2.524529,1.108405,2.635823,-0.189959,...,-2.260447,-1.929808,0.722827,0.501119,-0.180244,-1.944793,-1.604836,0.135558,0.9946,-1.378467


In [38]:
test_input = test_input.to_numpy()
reshaped_test_input = test_input.reshape(test_input.shape[0], 1, test_input.shape[1])
reshaped_test_input

array([[[ 0.56901595, -0.79909108, -1.58965928,  0.36860911,
         -0.76966101, -1.10113046, -2.52452908,  1.10840491,
          2.63582309, -0.18995911,  0.72640489,  2.66833308,
         -0.032632  ,  0.98802435,  0.24980494, -0.08955808,
          1.85434394,  2.02706568, -2.21150703,  1.42938591,
          0.85132668,  0.14092386,  0.30902324,  1.00267904,
          2.27970137,  1.11272593,  0.61007902,  0.89064538,
          1.42231438,  1.02650562,  1.47127593, -1.53201473,
          1.73503858,  1.66610613,  1.0456894 ,  0.55289259,
         -1.81216593, -2.10342296, -2.55377478,  0.41940628,
         -1.75756445,  0.241023  , -0.98659345,  1.52184916,
          1.01398607,  1.49290967,  1.40764092,  1.87930411,
         -2.51892072,  2.83356214,  1.58620924, -0.02524128,
          1.54231542,  1.5176119 ,  1.7355269 ,  1.87941982,
          1.54198129,  1.51276712,  1.83795594,  2.62210693,
          1.04993174,  0.96476493, -1.48277861, -2.26044738,
         -1.92980781,  0

In [39]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    4089.333
Name: Santa Catarina - Consumo de Cimento (t), dtype: float64

In [40]:
# Rede Neural Recorrente com optmizador Estocástico
def bidirectional_lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
        tf.keras.utils.set_random_seed(seed)

    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                                      patience=750, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True)
    
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(296, activation='tanh', return_sequences=True), 
                                      input_shape=(train_input.shape[1], train_input.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(148, activation='tanh', return_sequences=True)),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                        train_target, 
                        epochs=10000,
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [41]:
def get_a_good_seed(train_input, train_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = bidirectional_lstm_model(train_input, train_target, 0, seed)
        loss_list = history.history['loss'][500:]
        current_loss = min(loss_list)
        print(f"loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [42]:
winner_seed = get_a_good_seed(reshaped_train_input, train_target)
print(f"\n\nfinal_seed: {winner_seed}")

[2172498669, 2912802955, 3148364137, 816794283, 3071714403, 2448457290, 3465075628, 450866422, 1021442993, 2252424263, 3455374790, 4016494578, 3497972801, 4076484931, 3232972116, 3948113332, 3243035132, 1911734745, 3728353611, 1032001232, 1205550703, 3533056235, 2695769000, 187035824, 3365851479, 1308218483, 3456591574, 3511858, 1509837064, 1717383299, 3010464264, 2858720264, 4010219359, 1522395146, 2585312652, 1117043009, 4198809868, 2191457698, 3648920802, 677857919, 3939909741, 17537687, 1348937876, 1647973638, 149052683, 3221047859, 3441964510, 1537554186, 2645614473, 1311978601, 2531886764, 75400389, 1203709421, 3692316928, 2967499791, 2320587072, 1316335357, 3931409424, 973100885, 3938611169, 2171563848, 1935569001, 3335000028, 780905061, 1139337076, 1673764460, 2029560587, 3943593635, 2338712694, 2891100351, 5570889, 1172681210, 3431885502, 2944189219, 444329712, 3096104821, 3834127969, 4130307416, 3899404827, 4197326085, 3149724105, 2173476361, 156721076, 403706003, 2107745894,

2023-09-13 14:29:52.037534: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-13 14:29:52.070420: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-09-13 14:29:52.070563: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:995] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

loss: 9985.4013671875
winner_seed: 2172498669


Step: 1 ___________________________________________
loss: 12504.3115234375


Step: 2 ___________________________________________
loss: 15208.5380859375


Step: 3 ___________________________________________
loss: 7842.80712890625
winner_seed: 816794283


Step: 4 ___________________________________________
loss: 11526.58984375


Step: 5 ___________________________________________
loss: 17460.470703125


Step: 6 ___________________________________________
loss: 16037.4296875


Step: 7 ___________________________________________
loss: 15718.6064453125


Step: 8 ___________________________________________
loss: 12170.0859375


Step: 9 ___________________________________________
loss: 6716.36962890625
winner_seed: 2252424263


Step: 10 ___________________________________________
loss: 12110.5732421875


Step: 11 ___________________________________________
loss: 6479.35498046875
winner_seed: 4016494578


Step: 12 ________________________________

In [43]:
trained_model, history = bidirectional_lstm_model(reshaped_train_input, 
                                                  train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 3s 3s/step - loss: 9599418.0000
Epoch 2/10000
1/1 [==============================] - 0s 8ms/step - loss: 9389073.0000
Epoch 3/10000
1/1 [==============================] - 0s 10ms/step - loss: 6995174.5000
Epoch 4/10000
1/1 [==============================] - 0s 10ms/step - loss: 1220088.5000
Epoch 5/10000
1/1 [==============================] - 0s 10ms/step - loss: 90180.2891
Epoch 6/10000
1/1 [==============================] - 0s 6ms/step - loss: 182760.6250
Epoch 7/10000
1/1 [==============================] - 0s 5ms/step - loss: 104522.3828
Epoch 8/10000
1/1 [==============================] - 0s 4ms/step - loss: 121509.6875
Epoch 9/10000
1/1 [==============================] - 0s 5ms/step - loss: 90281.4297
Epoch 10/10000
1/1 [==============================] - 0s 9ms/step - loss: 64714.3594
Epoch 11/10000
1/1 [==============================] - 0s 10ms/step - loss: 55158.9883
Epoch 12/10000
1/1 [==============================] - 0s 5ms

1/1 [==============================] - 0s 5ms/step - loss: 35482.1836
Epoch 98/10000
1/1 [==============================] - 0s 5ms/step - loss: 57913.3555
Epoch 99/10000
1/1 [==============================] - 0s 5ms/step - loss: 62507.5273
Epoch 100/10000
1/1 [==============================] - 0s 5ms/step - loss: 59649.3438
Epoch 101/10000
1/1 [==============================] - 0s 5ms/step - loss: 39296.1055
Epoch 102/10000
1/1 [==============================] - 0s 6ms/step - loss: 31971.2285
Epoch 103/10000
1/1 [==============================] - 0s 5ms/step - loss: 48414.0469
Epoch 104/10000
1/1 [==============================] - 0s 5ms/step - loss: 38968.5586
Epoch 105/10000
1/1 [==============================] - 0s 5ms/step - loss: 47109.4688
Epoch 106/10000
1/1 [==============================] - 0s 5ms/step - loss: 22055.6855
Epoch 107/10000
1/1 [==============================] - 0s 5ms/step - loss: 45360.1445
Epoch 108/10000
1/1 [==============================] - 0s 5ms/step - los

1/1 [==============================] - 0s 4ms/step - loss: 84115.1172
Epoch 193/10000
1/1 [==============================] - 0s 4ms/step - loss: 55187.9414
Epoch 194/10000
1/1 [==============================] - 0s 5ms/step - loss: 29236.3535
Epoch 195/10000
1/1 [==============================] - 0s 5ms/step - loss: 60862.6445
Epoch 196/10000
1/1 [==============================] - 0s 4ms/step - loss: 45237.7031
Epoch 197/10000
1/1 [==============================] - 0s 5ms/step - loss: 24964.0449
Epoch 198/10000
1/1 [==============================] - 0s 4ms/step - loss: 46729.9258
Epoch 199/10000
1/1 [==============================] - 0s 4ms/step - loss: 49988.5273
Epoch 200/10000
1/1 [==============================] - 0s 4ms/step - loss: 44092.2656
Epoch 201/10000
1/1 [==============================] - 0s 4ms/step - loss: 54904.1602
Epoch 202/10000
1/1 [==============================] - 0s 4ms/step - loss: 37991.6289
Epoch 203/10000
1/1 [==============================] - 0s 4ms/step - l

1/1 [==============================] - 0s 5ms/step - loss: 39242.6016
Epoch 288/10000
1/1 [==============================] - 0s 4ms/step - loss: 37368.9336
Epoch 289/10000
1/1 [==============================] - 0s 8ms/step - loss: 15007.0107
Epoch 290/10000
1/1 [==============================] - 0s 5ms/step - loss: 36361.5664
Epoch 291/10000
1/1 [==============================] - 0s 4ms/step - loss: 38760.1875
Epoch 292/10000
1/1 [==============================] - 0s 4ms/step - loss: 48873.5039
Epoch 293/10000
1/1 [==============================] - 0s 4ms/step - loss: 60592.2969
Epoch 294/10000
1/1 [==============================] - 0s 4ms/step - loss: 49373.1875
Epoch 295/10000
1/1 [==============================] - 0s 4ms/step - loss: 33403.7031
Epoch 296/10000
1/1 [==============================] - 0s 4ms/step - loss: 22271.6875
Epoch 297/10000
1/1 [==============================] - 0s 4ms/step - loss: 55256.5352
Epoch 298/10000
1/1 [==============================] - 0s 4ms/step - l

1/1 [==============================] - 0s 4ms/step - loss: 39293.9297
Epoch 383/10000
1/1 [==============================] - 0s 4ms/step - loss: 13548.8955
Epoch 384/10000
1/1 [==============================] - 0s 4ms/step - loss: 27593.0059
Epoch 385/10000
1/1 [==============================] - 0s 5ms/step - loss: 37221.9219
Epoch 386/10000
1/1 [==============================] - 0s 4ms/step - loss: 26022.7656
Epoch 387/10000
1/1 [==============================] - 0s 4ms/step - loss: 35027.1953
Epoch 388/10000
1/1 [==============================] - 0s 4ms/step - loss: 47932.0117
Epoch 389/10000
1/1 [==============================] - 0s 4ms/step - loss: 36045.1680
Epoch 390/10000
1/1 [==============================] - 0s 4ms/step - loss: 22525.0918
Epoch 391/10000
1/1 [==============================] - 0s 4ms/step - loss: 37629.6406
Epoch 392/10000
1/1 [==============================] - 0s 6ms/step - loss: 25034.7832
Epoch 393/10000
1/1 [==============================] - 0s 5ms/step - l

...

1/1 [==============================] - 0s 4ms/step - loss: 43881.4375
Epoch 1799/10000
1/1 [==============================] - 0s 4ms/step - loss: 66786.1797
Epoch 1800/10000
1/1 [==============================] - 0s 4ms/step - loss: 25673.2891
Epoch 1801/10000
1/1 [==============================] - 0s 4ms/step - loss: 61399.7695
Epoch 1802/10000
1/1 [==============================] - 0s 4ms/step - loss: 55505.8633
Epoch 1803/10000
1/1 [==============================] - 0s 4ms/step - loss: 46365.5039
Epoch 1804/10000
1/1 [==============================] - 0s 4ms/step - loss: 50808.0000
Epoch 1805/10000
1/1 [==============================] - 0s 4ms/step - loss: 49685.9648
Epoch 1806/10000
1/1 [==============================] - 0s 4ms/step - loss: 64554.6992
Epoch 1807/10000
1/1 [==============================] - 0s 4ms/step - loss: 38148.3477
Epoch 1808/10000
1/1 [==============================] - 0s 4ms/step - loss: 30311.0410
Epoch 1809/10000
1/1 [==============================] - 0s 4

1/1 [==============================] - 0s 6ms/step - loss: 30350.1094
Epoch 1893/10000
1/1 [==============================] - 0s 5ms/step - loss: 45009.3438
Epoch 1894/10000
1/1 [==============================] - 0s 4ms/step - loss: 26124.9434
Epoch 1895/10000
1/1 [==============================] - 0s 5ms/step - loss: 28413.8262
Epoch 1896/10000
1/1 [==============================] - 0s 4ms/step - loss: 71978.0234
Epoch 1897/10000
1/1 [==============================] - 0s 5ms/step - loss: 49452.9062
Epoch 1898/10000
1/1 [==============================] - 0s 5ms/step - loss: 47866.3125
Epoch 1899/10000
1/1 [==============================] - 0s 4ms/step - loss: 39173.3477
Epoch 1900/10000
1/1 [==============================] - 0s 5ms/step - loss: 22367.2910
Epoch 1901/10000
1/1 [==============================] - 0s 6ms/step - loss: 24984.1934
Epoch 1902/10000
1/1 [==============================] - 0s 4ms/step - loss: 45539.5508
Epoch 1903/10000
1/1 [==============================] - 0s 4

1/1 [==============================] - 0s 6ms/step - loss: 41265.0312
Epoch 1987/10000
1/1 [==============================] - 0s 8ms/step - loss: 52249.5039
Epoch 1988/10000
1/1 [==============================] - 0s 9ms/step - loss: 57015.1406
Epoch 1989/10000
1/1 [==============================] - 0s 10ms/step - loss: 62865.6211
Epoch 1990/10000
1/1 [==============================] - 0s 7ms/step - loss: 44630.8555
Epoch 1991/10000
1/1 [==============================] - 0s 6ms/step - loss: 22242.0215
Epoch 1992/10000
1/1 [==============================] - 0s 7ms/step - loss: 54832.1992
Epoch 1993/10000
1/1 [==============================] - 0s 5ms/step - loss: 78941.4453
Epoch 1994/10000
1/1 [==============================] - 0s 4ms/step - loss: 29026.5293
Epoch 1995/10000
1/1 [==============================] - 0s 6ms/step - loss: 43200.0820
Epoch 1996/10000
1/1 [==============================] - 0s 5ms/step - loss: 25880.0059
Epoch 1997/10000
1/1 [==============================] - 0s 

1/1 [==============================] - 0s 5ms/step - loss: 23499.1660
Epoch 2081/10000
1/1 [==============================] - 0s 4ms/step - loss: 68351.7578
Epoch 2082/10000
1/1 [==============================] - 0s 4ms/step - loss: 30713.9062
Epoch 2083/10000
1/1 [==============================] - 0s 6ms/step - loss: 25728.8691
Epoch 2084/10000
1/1 [==============================] - 0s 4ms/step - loss: 50243.3477
Epoch 2085/10000
1/1 [==============================] - 0s 6ms/step - loss: 38031.5938
Epoch 2086/10000
1/1 [==============================] - 0s 6ms/step - loss: 36910.9102
Epoch 2087/10000
1/1 [==============================] - 0s 7ms/step - loss: 39812.1094
Epoch 2088/10000
1/1 [==============================] - 0s 6ms/step - loss: 56793.5117
Epoch 2089/10000
1/1 [==============================] - 0s 5ms/step - loss: 37487.5430
Epoch 2090/10000
1/1 [==============================] - 0s 5ms/step - loss: 39222.7148
Epoch 2091/10000
1/1 [==============================] - 0s 5

1/1 [==============================] - 0s 4ms/step - loss: 33112.8867
Epoch 2175/10000
1/1 [==============================] - 0s 4ms/step - loss: 23653.9531
Epoch 2176/10000
1/1 [==============================] - 0s 4ms/step - loss: 53292.9883
Epoch 2177/10000
1/1 [==============================] - 0s 4ms/step - loss: 53043.9219
Epoch 2178/10000
1/1 [==============================] - 0s 5ms/step - loss: 91277.7734
Epoch 2179/10000
1/1 [==============================] - 0s 4ms/step - loss: 60249.6445
Epoch 2180/10000
1/1 [==============================] - 0s 4ms/step - loss: 59634.5977
Epoch 2181/10000
1/1 [==============================] - 0s 4ms/step - loss: 45007.5117
Epoch 2182/10000
1/1 [==============================] - 0s 5ms/step - loss: 34185.0977
Epoch 2183/10000
1/1 [==============================] - 0s 4ms/step - loss: 60906.5664
Epoch 2184/10000
1/1 [==============================] - 0s 4ms/step - loss: 52452.5508
Epoch 2185/10000
1/1 [==============================] - 0s 6

In [44]:
prediction = trained_model.predict(reshaped_test_input)[0][0]
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 1s 706ms/step
[4089.333](test_target) - [3364.2979](prediction) = 725.0351484375001


In [45]:
porcentage = mean_absolute_error(test_target, prediction)/test_target.values
porcentage[0]

0.1772991215040448

In [46]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = bidirectional_lstm_model(dfToInputRNN(train_input), 
                                                     train_target, want_verbose=0, seed=seed)

            prediction = model.predict(dfToInputRNN(test_input))[0][0]

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [47]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado,Santa Catarina - Desemprego,Santa Catarina - IDH,Santa Catarina - IDH Longevidade,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Santa Catarina - IDH Educacao,Santa Catarina - IDH Renda
0,-0.932986,-1.099975,-0.729604,-0.816011,-0.465381,-1.243762,-1.312210,1.148205,-1.122043,-1.121418,...,1.412346,1.323311,1.276699,0.891356,0.813979,-0.083359,-1.191078,0.945270,-1.234276,1.299312
1,-0.453916,-0.219779,-0.684368,-0.592292,1.389222,0.038963,0.199402,0.140893,-0.184474,-0.185492,...,-0.643250,-0.229640,-1.165161,-1.396527,0.594549,1.264295,1.255842,0.438326,0.019291,-0.166085
2,1.386902,1.319754,1.413972,1.408303,-0.923841,1.204798,1.112809,-1.289098,1.306518,1.306910,...,-0.769096,-1.093670,-0.111538,0.505170,-1.408528,-1.180936,-0.064763,-1.383595,1.214985,-1.133227


train_target:


,Santa Catarina - Consumo de Cimento (t)
0,2669.400
1,2852.759
2,3087.810


test_input:


...

,Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado,Santa Catarina - Desemprego,Santa Catarina - IDH,Santa Catarina - IDH Longevidade,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Taxa Selic (%),Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Santa Catarina - IDH Educacao,Santa Catarina - IDH Renda
12,0.569016,-0.799091,-1.589659,0.368609,-0.769661,-1.10113,-2.524529,1.108405,2.635823,-0.189959,...,-2.260447,-1.929808,0.722827,0.501119,-0.180244,-1.944793,-1.604836,0.135558,0.9946,-1.378467


test_target:


,Santa Catarina - Consumo de Cimento (t)
12,4089.333


1/1 [==============================] - 1s 571ms/step
Error: 725.0351484375001




[3037.7099609375,
 3215.88134765625,
 3395.48681640625,
 2955.317138671875,
 2939.111083984375,
 2885.34130859375,
 2800.720703125,
 2848.808349609375,
 3130.2333984375,
 3364.2978515625]

In [48]:
display(targets)
display(predictions)

[3268.31,
 3466.227,
 3246.749285714286,
 3017.084228571429,
 2776.217485714286,
 2891.489,
 2976.485,
 3258.027,
 3537.032,
 4089.333]

[3037.7099609375,
 3215.88134765625,
 3395.48681640625,
 2955.317138671875,
 2939.111083984375,
 2885.34130859375,
 2800.720703125,
 2848.808349609375,
 3130.2333984375,
 3364.2978515625]

In [49]:
mae = mean_absolute_error(predictions, targets)
mae

257.73082989397324

In [50]:
porcentage = mae/np.mean(targets)
porcentage

0.0792360790666022